<a href="https://colab.research.google.com/github/Azaharuddin786/MY-FIRST-PROJECT-786/blob/main/yolov1model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from dataset import VOCDataset
from utils import(
    intersection_over_union,
    non_max_suppression,
    mean_average_precision,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss import YoloLoss


In [2]:
seed = 123
torch.manual_seed(seed)

In [3]:
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 1000
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "/content/drive/MyDrive/Test_Data/images"
LABEL_DIR = "/content/drive/MyDrive/Test_Data/labels"

In [4]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes

In [5]:
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

In [6]:
def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item())

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")

In [7]:

model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
loss_fn = YoloLoss()

if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

In [8]:
  train_dataset = VOCDataset(
        "/content/100examples.csv",
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR,
    )


In [9]:
 test_dataset = VOCDataset(
        "/content/test.csv", transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,
    )

In [10]:
 train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

In [11]:
  test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

In [12]:
for epoch in range(EPOCHS):
  pred_boxes,target_boxes = get_bboxes(
      train_loader,model,iou_threshold=0.5,threshold=0.4
  )
  mean_avg_prec = mean_average_precision(
      pred_boxes,target_boxes,iou_threshold=0.5,box_format="midpoint"
  )
  print(f"Train mAP: {mean_avg_prec}")
  train_fn(train_loader, model, optimizer, loss_fn)

Train mAP: 0.0


100%|██████████| 6/6 [00:05<00:00,  1.03it/s, loss=968]

Mean loss was 939.6077473958334


Train mAP: 0.0


100%|██████████| 6/6 [00:04<00:00,  1.22it/s, loss=421]

...

100%|██████████| 6/6 [00:04<00:00,  1.22it/s, loss=2.85]

Mean loss was 3.2964195013046265


Train mAP: 0.9971329569816589


100%|██████████| 6/6 [00:04<00:00,  1.23it/s, loss=2.82]

Mean loss was 2.831403613090515


In [ ]:
if __name__ == "__main__":
    main()